## libraries

In [8]:
import openai
from functools import cache

## Step 1: Use RAG, query the database and give additional information to make the prompt better. (aka APES textbook) 

folded

## Step 2: Initial LLM output: 
Primer: tell it that its an educational bot and it can wrap things in image tags to draw images as well. Include an example input and example output. 

Additional context: this, you will get from step 1. 


In [9]:
primer ='''
You are an automatic wikihow bot website creator where I feed you a prompt and you generate all of the steps in order to achieve this prompt. DO insert a couple of images to show the user how to do the task. 

1. Take that task, devise 3-8 detailed, separate steps on how to complete the task.
2. Give me an exact prompt to give to a model that outputs an ai generated image, given a descriptive text prompt outlining every precise detail of the desired image. Each prompt per step should be 50 words or less, so be descriptive but concise. Surround the decsription tag with <image> tags.

ONLY respond with text, DO NOT respond with markdown or other formatting. 

EXAMPLE USER QUERY: 
“I want to learn how to ride a bike".

EXAMPLE OUTPUT: 
How to Ride a Bike

Step 1: Choose the Right Bike
Pick a bike that fits your height and comfort. Your feet should touch the ground when seated.
<image> Image of different bike sizes and a person measuring for fit. <image>

Step 2: Find a Suitable Area
Start in a flat, open area, like a park, away from traffic.
<image> Image of an open park with a marked biking area. <image>

Step 3: Wear Safety Gear
Put on a helmet and protective pads for safety.
<image> Image of a person wearing a helmet, elbow pads, and knee pads. <image>

Step 4: Mount the Bike
Stand next to the bike, grip the handles, and swing one leg over to sit.
<image> Image of a person mounting a bike step-by-step. <image>

Step 5: Find Your Balance
Push off with one foot and coast, placing feet on the pedals only when you've found balance.
<image> Image of a person coasting and balancing on a bike. <image>

Step 6: Start Pedaling
Begin pedaling slowly. Look forward, keeping hands on the handlebars.
<image> Image of a person pedaling and moving forward. <image>

Step 7: Practice Steering
Turn the handlebars gently in the direction you want to go.
<image> Image of hands turning bike handlebars. <image>

Step 8: Learn to Brake
Familiarize yourself with both front and rear brakes. Press them gently to slow down or stop.
<image> Image of hands squeezing bike brakes. <image>

Step 9: Ride Regularly
Practice often to build confidence and improve skills.
<image> Image of a person riding through various terrains. <image>
'''


@cache
def get_gpt_response(query: str):
    augmented_query = f'{query}\n\n"OUTPUT: "'
    res = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": primer},
            {"role": "user", "content": augmented_query}
        ]
    )
    return res['choices'][0]['message']['content']


In [55]:
import together
from functools import cache
import base64
from PIL import Image
from io import BytesIO
from IPython.display import display
from random import randint

together.api_key = 'dd6fc37975f3b9e88b25fa63b1ddd22ad6043a4449c39d061ffc6c8117a85f33'

@cache
def description_to_image(description: str, index: int):
    model_list = together.Models.list()

    output = together.Image.create(
        prompt = description + ' in color', 
        model = "SG161222/Realistic_Vision_V3.0_VAE",
        width = 512, 
        height = 512,
        steps = 20,
        seed = randint(0, 100)
    )
    
    base64_string = output['output']['choices'][0]['image_base64']
    image_data = base64.b64decode(base64_string)
    image_buffer = BytesIO(image_data)
    image = Image.open(image_buffer)
    image.save(f'images/{index}.png')

In [56]:
# query = 'I want to learn how to change a tire' 
query = 'tour guide on paris'

res = get_gpt_response(query)
for idx, description in enumerate(res.split('<image>')[1::2]):
    description_to_image(description, idx)

In [29]:
description

' Image of a person reading a book or studying about Parisian history and culture. '

In [59]:
print(res)

How to Be a Tour Guide in Paris

Step 1: Learn the History and Culture
Study the rich history and vibrant culture of Paris, including famous landmarks, art, and cuisine.
<image> Image of a person reading a book or studying about Parisian history and culture. <image>

Step 2: Know the Top Attractions
Familiarize yourself with popular tourist destinations, such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.
<image> Image of a person holding a map with marked locations of top attractions in Paris. <image>

Step 3: Master the Language
Learn basic French phrases to communicate with both locals and tourists. Practice pronunciation and common expressions.
<image> Image of a person using language learning resources or talking to a language tutor. <image>

Step 4: Develop a Route
Plan an efficient and engaging tour route that covers major landmarks and offers interesting insights and facts.
<image> Image of a person using a map and plotting a tour route with important stops in Pa

In [83]:
text = get_gpt_response(query)

steps = text.split(':')
title = steps[0].split('\n')[0]

for idx, step in enumerate(steps[1:]):
    pair = step.split("Step")[0].split('<image>')
    print(f'Step {idx+1} - instructions: {pair[0]} - image description: {pair[1]}')


Step 1 -  Learn the History and Culture
Study the rich history and vibrant culture of Paris, including famous landmarks, art, and cuisine.
 -  Image of a person reading a book or studying about Parisian history and culture. 
Step 2 -  Know the Top Attractions
Familiarize yourself with popular tourist destinations, such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.
 -  Image of a person holding a map with marked locations of top attractions in Paris. 
Step 3 -  Master the Language
Learn basic French phrases to communicate with both locals and tourists. Practice pronunciation and common expressions.
 -  Image of a person using language learning resources or talking to a language tutor. 
Step 4 -  Develop a Route
Plan an efficient and engaging tour route that covers major landmarks and offers interesting insights and facts.
 -  Image of a person using a map and plotting a tour route with important stops in Paris. 
Step 5 -  Provide Interesting Facts and Stories
Research ca

'How to Be a Tour Guide in Paris\n\nStep 1'